In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import scipy
from scipy import stats
from scipy.stats import skew, kurtosis
import os

#Function to create individual csv files
def CreateFile(folder_Name, sub_folder_Name, data_Frame): 
    file_Name= os.path.join(folder_Name,sub_folder_Name);
    file_Check= file_Name + ".csv"
    
    #Create new csv file for each valveNr_WorkpieceNr
    try:
        data_Frame.to_csv(file_Check, encoding='utf-8', index=True)
        return file_Check,folder_Name, sub_folder_Name
    except:
        print("error in creating csv file");  
        
basepath = r"D:\Predictions\MongoData"; #Basepath where the folders/files are created
toolFile = ["17015_final", "17017_final"] #Control which tools

def PrepareData(Integrated_folders):  
 ValveNr_WorkpieceNr = [f.path for f in os.scandir(basepath) if f.is_dir()] #Gets list of workpiece and valve number folders
 V_N = [f.name  for f in os.scandir(basepath) if f.is_dir()]
 completeData = []
 Vr_Nr = []
 ValveNrList = []
    
 for i in range(0,len(ValveNr_WorkpieceNr)): 
    folderCount = 0;
    data = {}    
    for j in range(0, len(toolFile)):
     basePath2 = os.path.join(ValveNr_WorkpieceNr[i],toolFile[j]); 
     if os.path.isdir(basePath2):
      folderCount+=1;
    
    if folderCount == len(toolFile):  
     for k in range(0, len(toolFile)):
      basePath2 = os.path.join(ValveNr_WorkpieceNr[i],toolFile[k]);
      Value = [v.path for v in os.scandir(basePath2) if v.is_file()] 
      
      Values = [Value[1]] ##Tool Values (csv files)
      for l in range(0, len(Values)):  
       data[str(toolFile[k])+'_value'+str(l+1)+'_mean'] = []
       data[str(toolFile[k])+'_value'+str(l+1)+'_SD'] = []
       data[str(toolFile[k])+'_value'+str(l+1)+'_Kurtosis'] = []
       data[str(toolFile[k])+'_value'+str(l+1)+'_Skewness'] = []
       
       x = pd.read_csv(Values[l], encoding = 'ISO-8859-1', sep=','); 
       x = abs(x.iloc[:,1:]); #Remove first column (numbers)
    
       #Calculating features
       y = x.mean(axis=0); #Column mean 
       z = x.std(axis=0); #Column standard deviation 
       ku = scipy.stats.kurtosis(x, fisher=True, bias=True) #Column kurtosis; Using Fisher Kurtosis
                                                            #3.0 is subtracted from the result to give 0.0 for a normal distribution.
       sk = scipy.stats.skew(x, bias=True) #Column skewness 
               
       data[str(toolFile[k])+'_value'+str(l+1)+'_mean'].append(y['Data']);
       data[str(toolFile[k])+'_value'+str(l+1)+'_SD'].append(z['Data']);
       data[str(toolFile[k])+'_value'+str(l+1)+'_Kurtosis'].append(ku[0]);
       data[str(toolFile[k])+'_value'+str(l+1)+'_Skewness'].append(sk[0]); 
        
     Vr_Nr.append(V_N[i]); 
     df = pd.DataFrame(data, index=[V_N[i]]);
     df.replace(np.nan, 0 , regex=True,inplace=True)
        
     CreateFile(ValveNr_WorkpieceNr[i], V_N[i], df); #Create csv file for each valveNr_WorkpieceNr combination
     completeData.append(df);
     df1 = pd.concat(completeData, axis=0)
    CreateFile(basepath,'Extracted_features', df1); #Create one csv file for all valveNr_WorkpieceNr combinations
    Checkpredictions(Vr_Nr) #Start predicting diameter with the trained model #Calling function
    
def Checkpredictions(Valve_WP_Nr=0):
    fea = pd.read_csv(basepath+'\Extracted_features.csv', encoding = 'ISO-8859-1')
    fea = fea.T[1:17].T  # same features as for training the model

    #Depending on the Workpiece Valve Number, call the trained model
    #Remember, the model has to be trained in the same environment as the machining PC (where the code is running)
    localValve = []
    localValve.append(Valve_WP_Nr[0].split("_")[0])
    with open(r'C:\Users\Rexroth\Nikita\Final_code_with_GUI\Automation_curr\\'+localValve[0]+'_finalized_trained_RFR_model_for_diameter_prediction.sav', 'rb') as file:
     rfr = pickle.load(file)
    predictions= rfr.predict(fea)
    predictions = np.around(predictions,decimals=4)
    pred = pd.DataFrame(predictions, columns=['Predictions'])
    pred['WorkpieceNumber'] = Valve_WP_Nr;
    CreateFile(basepath,'Predictions', pred);  #Keep updating one csv file with all the diameter prediction values
                                               #for all valveNr_WorkpieceNr combinations     
    

--- Logging error ---
Traceback (most recent call last):
  File "C:\Program Files\Anaconda3\lib\logging\__init__.py", line 992, in emit
    msg = self.format(record)
  File "C:\Program Files\Anaconda3\lib\logging\__init__.py", line 838, in format
    return fmt.format(record)
  File "C:\Program Files\Anaconda3\lib\logging\__init__.py", line 575, in format
    record.message = record.getMessage()
  File "C:\Program Files\Anaconda3\lib\logging\__init__.py", line 338, in getMessage
    msg = msg % self.args
TypeError: %d format: a number is required, not list
Call stack:
  File "C:\Program Files\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Program Files\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Program Files\Anaconda3\lib\site-packages\traitlets\config\application.py", l

TypeError: unsupported operand type(s) for +: 'int' and 'str'